In [1]:
import pandas as pd
import numpy as np

from keras.layers import Dense, Activation, Dropout
from keras.models import Sequential
from keras.models import load_model
import keras.backend as K

from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold

import seaborn as sns
%pylab inline

C:\Users\cosin\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Populating the interactive namespace from numpy and matplotlib


In [10]:
put_data = pd.read_csv('Putin1.csv', delimiter=';', names=range(4000))
nav_data = pd.read_csv('Navalny1.csv', delimiter=';', names=range(4000))
put_test = pd.read_csv('Putin_test.csv', delimiter=';', names=range(4000))
nav_test = pd.read_csv('Navalny_test.csv', delimiter=';', names=range(4000))

In [64]:
put_num = put_data.shape[0]
put_num

6760

In [65]:
nav_num = nav_data.shape[0]
nav_num

1957

In [17]:
put_test_num = put_test.shape[0]
put_test_num

969

In [18]:
nav_test_num = nav_test.shape[0]
nav_test_num

538

# Training data

In [19]:
data = pd.concat((put_data.iloc[:1957,:], nav_data))

In [20]:
data.shape

(3914, 4000)

In [21]:
data.dropna(axis=1, how='all', inplace=True)
data.fillna(0, inplace=True) # replacing NaNs with zeros
data = data.applymap(int)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,1722,1723,1724,1725,1726,1727,1728,1729,1730,1731
0,473365518,20,2,-1,-50498710,-49606842,-48928605,-16389388,0,0,...,0,0,0,0,0,0,0,0,0,0
1,800930,36,2,-1,-106564427,-9758,-160937227,-26419239,-70365191,-25653235,...,0,0,0,0,0,0,0,0,0,0
2,346505303,-1,2,-1,-131188697,-64299218,-123775571,-106823150,-61484302,-151336239,...,0,0,0,0,0,0,0,0,0,0
3,307080832,21,2,-1,-43776317,-57846937,-62965884,-70870022,-23951627,-124305154,...,0,0,0,0,0,0,0,0,0,0
4,404314502,-1,2,5,-113801785,-6726778,-17568841,-94686473,-159187081,-161812969,...,0,0,0,0,0,0,0,0,0,0


In [25]:
publics = np.unique(data.iloc[:, 4:].values) # list of publics
publics.shape

(127235,)

In [26]:
publics = np.delete(publics, np.where(publics == 0)[0]) # deleting id=0
publics.shape

(127234,)

In [27]:
counters = np.zeros(publics.shape[0])
counters.shape

(127234,)

In [28]:
genders = np.unique(data.iloc[:, 2]) # list of genders
genders

array([-1,  0,  1,  2], dtype=int64)

In [29]:
preferences = np.unique(data.iloc[:, 3]) # list of political preferences
preferences

array([-1,  1,  2,  3,  4,  5,  6,  7,  8,  9], dtype=int64)

In [32]:
users_num = data.shape[0] # number of users in dataset
users_num

3914

In [33]:
Ages=[0, 18, 26, 36, 51, 66, 86]

In [34]:
X_ages = np.zeros([users_num, len(Ages)+1], dtype=np.float16)
X_sex = np.zeros([users_num, genders.shape[0]], dtype=np.float16)
X_pref = np.zeros([users_num, preferences.shape[0]], dtype=np.float16)

In [35]:
for i in range(users_num):
    print(str(i)+'/'+str(users_num), end='\r')
    age = data.iloc[i, 1]
    sex = data.iloc[i, 2]
    pref = data.iloc[i, 3]
    publ = data.iloc[i, 4:]
    
    for a in range(len(Ages)): # filling X_ages
        if age < Ages[a]:
            X_ages[i, a] = 1
            break
        elif age >= Ages[len(Ages)-1]:
            X_ages[i, len(Ages)] = 1
            break
    
    for j in range(genders.shape[0]): # filling genders
        if sex == genders[j]:
            X_sex[i, j] = 1
    
    for j in range(preferences.shape[0]): # filling preferences
        if pref == preferences[j]:
            X_pref[i, j] = 1
    
    for j in publ: # counting the number of occurence of each public
        if j == 0:
            break
        publ_index = np.where(publics == j)[0][0]
        counters[publ_index] = counters[publ_index] + 1

In [37]:
def activaton(ind): # for taking into account the order of publics
    return 1/(1+np.exp(0.2*ind)) + 0.5

In [38]:
X_publ = []
X_publ = np.zeros([users_num, publics.shape[0]], dtype=np.float16)
for i in range(users_num):
    print(str(i)+'/'+str(users_num), end='\r')
    publ = data.iloc[i, 4:].values
    for j in range(publ.shape[0]):
        if publ[j] == 0:
            break
        publ_index = np.where(publics == publ[j])[0]
        X_publ[i, publ_index] = activaton(j)

In [39]:
X = np.hstack((X_ages, X_sex, X_pref, X_publ)) # merging all arrays into one
X.shape

(3914, 127256)

In [76]:
# PCA components
n_components = 14

In [41]:
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42) # cross-validation

In [42]:
pca = PCA(n_components=n_components)
pca.fit(X)

PCA(copy=True, iterated_power='auto', n_components=14, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [43]:
from sklearn.externals import joblib # saving pca
_ = joblib.dump(pca, 'pcaPutNav.save')

In [44]:
X_pca = pca.transform(X)

C:\Users\cosin\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)


# Test data

In [51]:
data_test = pd.concat((put_test, nav_test))
num = data_test.shape[0]
num

1507

In [52]:
data_test.dropna(axis=1, how='all', inplace=True)
data_test.fillna(0, inplace=True) # replacing NaNs with zeros
data_test = data_test.applymap(int)
data_test.head()

,0,1,2,3,4,5,6,7,8,9,...,194,195,196,197,198,199,200,201,202,203
0,330864174,-1,2,-1,-58798248,-35145657,-57846937,-83876626,-84429649,-60397113,...,0,0,0,0,0,0,0,0,0,0
1,217809018,28,2,-1,-142754704,-66621324,-70204174,-54154078,-77963997,-23064236,...,0,0,0,0,0,0,0,0,0,0
2,20671231,30,2,-1,-133742557,-40836944,-22798006,-60443553,-88759692,-125303834,...,0,0,0,0,0,0,0,0,0,0
3,366089932,15,1,-1,-116871226,-133791433,-83985614,-126506885,-142918020,-122181832,...,0,0,0,0,0,0,0,0,0,0
4,399763476,-1,2,-1,-39399586,-135646310,-58170807,-100954660,-15326149,-111205860,...,0,0,0,0,0,0,0,0,0,0


In [53]:
X_ages = np.zeros([num, len(Ages)+1], dtype=np.float16)
X_sex = np.zeros([num, genders.shape[0]], dtype=np.float16)
X_pref = np.zeros([num, preferences.shape[0]], dtype=np.float16)
X_publ = np.zeros([num, publics.shape[0]], dtype=np.float16)

In [54]:
for i in range(num):
    print(str(i)+'/'+str(num), end='\r')
    age = data_test.iloc[i, 1]
    sex = data_test.iloc[i, 2]
    pref = data_test.iloc[i, 3]
    publ = data_test.iloc[i, 4:]
    
    for a in range(len(Ages)): # filling X_ages
        if age < Ages[a]:
            X_ages[i, a] = 1
            break
        elif age >= Ages[len(Ages)-1]:
            X_ages[i, len(Ages)] = 1
            break
    
    for j in range(genders.shape[0]): # filling genders
        if sex == genders[j]:
            X_sex[i, j] = 1
    
    for j in range(preferences.shape[0]): # filling preferences
        if pref == preferences[j]:
            X_pref[i, j] = 1
    
    for j in range(publ.shape[0]):
        if publ.values[j] == 0:
            break
        publ_index = np.where(publics == publ.values[j])[0]
        X_publ[i, publ_index] = activaton(j)

In [55]:
X_test = np.hstack((X_ages, X_sex, X_pref, X_publ)) # merging all arrays into one
X_test.shape

(1507, 127256)

In [56]:
X_test_pca = pca.transform(X_test)

In [62]:
X_test_pca.shape

(1507, 14)

In [57]:
np.save('trainPutNavPCA.npy', X_pca)
np.save('testPutNavPCA.npy', X_test_pca)

In [67]:
y = np.append(np.zeros(nav_num), np.ones(nav_num))
y_test = np.append(np.zeros(put_test_num), np.ones(nav_test_num))

In [68]:
y.shape

(3914,)

In [127]:
# NN hyperparameters
batch_size = 600
epochs = 200
dropout_size = 0.5

# hidden layers dimensions
hidden_first = 50
hidden_second = 25
hidden_third = 5

In [128]:
model = Sequential()
model.add(Dense(hidden_first, input_dim=n_components, activation='relu'))
Dropout(dropout_size)
model.add(Dense(hidden_second, activation='relu'))
Dropout(dropout_size/2)
model.add(Dense(hidden_third, activation='relu'))
Dropout(dropout_size/3)
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_pca, y, batch_size=batch_size, epochs=epochs, shuffle=True, 
          validation_data=(X_test_pca[430:, :], y_test[430:]))

Acc = []
Tpr = []
Tnr = []
Ppv = []
Npv = []
F1 = []

y_pred = model.predict(X_test_pca[430:, :]).round().reshape(-1)
tp = (y_pred * y_test[430:]).sum() # predicted - 1, actual - 1
tn = ((1-y_pred) * (1-y_test[430:])).sum() # predicted - 0, actual - 0
fp = (y_pred * (1-y_test[430:])).sum() # predicted - 1, actual - 0
fn = ((1-y_pred) * y_test[430:]).sum() # predicted - 0, actual - 1

acc = (tp + tn) / (tp + tn + fp + fn)
tpr = tp / (tp + fn)
tnr = tn / (tn + fp)
ppv = tp / (tp + fp)
npv = tn / (tn + fn)
f1 = (2 * tpr * ppv) / (tpr + ppv)

Acc.append(int(acc * 1000) / 10.0)
Tpr.append(int(tpr * 1000) / 10.0)
Tnr.append(int(tnr * 1000) / 10.0)
Ppv.append(int(ppv * 1000) / 10.0)
Npv.append(int(npv * 1000) / 10.0)
F1.append(int(f1 * 1000) / 1000.0)

Acc = np.array(Acc).astype(float)
Tpr = np.array(Tpr).astype(float)
Tnr = np.array(Tnr).astype(float)
Ppv = np.array(Ppv).astype(float)
Npv = np.array(Npv).astype(float)
F1 = np.array(F1).astype(float)

Train on 3914 samples, validate on 1077 samples
Epoch 1/200
3914/3914 [==============================] - 5s 1ms/step - loss: 0.6706 - acc: 0.6022 - val_loss: 0.6754 - val_acc: 0.5590
Epoch 2/200
3914/3914 [==============================] - 0s 25us/step - loss: 0.6466 - acc: 0.6903 - val_loss: 0.6609 - val_acc: 0.6110
Epoch 3/200
3914/3914 [==============================] - 0s 23us/step - loss: 0.6142 - acc: 0.7976 - val_loss: 0.6422 - val_acc: 0.6453
Epoch 4/200
3914/3914 [==============================] - 0s 20us/step - loss: 0.5686 - acc: 0.8679 - val_loss: 0.6191 - val_acc: 0.6713
Epoch 5/200
3914/3914 [==============================] - 0s 22us/step - loss: 0.5138 - acc: 0.9144 - val_loss: 0.5936 - val_acc: 0.6908
Epoch 6/200
3914/3914 [==============================] - 0s 24us/step - loss: 0.4550 - acc: 0.9280 - val_loss: 0.5698 - val_acc: 0.7038
Epoch 7/200
3914/3914 [==============================] - 0s 24us/step - loss: 0.3963 - acc: 0.9325 - val_loss: 0.5505 - val_acc: 0.7112
E

3914/3914 [==============================] - 0s 24us/step - loss: 0.0989 - acc: 0.9645 - val_loss: 0.5870 - val_acc: 0.8050
Epoch 61/200
3914/3914 [==============================] - 0s 26us/step - loss: 0.0986 - acc: 0.9640 - val_loss: 0.5920 - val_acc: 0.8041
Epoch 62/200
3914/3914 [==============================] - 0s 27us/step - loss: 0.0987 - acc: 0.9642 - val_loss: 0.5879 - val_acc: 0.8050
Epoch 63/200
3914/3914 [==============================] - 0s 24us/step - loss: 0.0983 - acc: 0.9645 - val_loss: 0.6000 - val_acc: 0.7994
Epoch 64/200
3914/3914 [==============================] - 0s 25us/step - loss: 0.0978 - acc: 0.9650 - val_loss: 0.5857 - val_acc: 0.8041
Epoch 65/200
3914/3914 [==============================] - 0s 24us/step - loss: 0.0975 - acc: 0.9650 - val_loss: 0.5842 - val_acc: 0.8069
Epoch 66/200
3914/3914 [==============================] - 0s 24us/step - loss: 0.0974 - acc: 0.9653 - val_loss: 0.5889 - val_acc: 0.8050
Epoch 67/200
3914/3914 [==============================

Epoch 120/200
3914/3914 [==============================] - 0s 21us/step - loss: 0.0863 - acc: 0.9693 - val_loss: 0.5716 - val_acc: 0.8162
Epoch 121/200
3914/3914 [==============================] - 0s 22us/step - loss: 0.0864 - acc: 0.9691 - val_loss: 0.5912 - val_acc: 0.8134
Epoch 122/200
3914/3914 [==============================] - 0s 23us/step - loss: 0.0861 - acc: 0.9683 - val_loss: 0.5842 - val_acc: 0.8152
Epoch 123/200
3914/3914 [==============================] - 0s 22us/step - loss: 0.0864 - acc: 0.9699 - val_loss: 0.5711 - val_acc: 0.8189
Epoch 124/200
3914/3914 [==============================] - 0s 22us/step - loss: 0.0861 - acc: 0.9683 - val_loss: 0.6008 - val_acc: 0.8106
Epoch 125/200
3914/3914 [==============================] - 0s 27us/step - loss: 0.0853 - acc: 0.9693 - val_loss: 0.5661 - val_acc: 0.8180
Epoch 126/200
3914/3914 [==============================] - 0s 26us/step - loss: 0.0856 - acc: 0.9699 - val_loss: 0.5859 - val_acc: 0.8162
Epoch 127/200
3914/3914 [=========

3914/3914 [==============================] - 0s 22us/step - loss: 0.0779 - acc: 0.9732 - val_loss: 0.6179 - val_acc: 0.8152
Epoch 180/200
3914/3914 [==============================] - 0s 22us/step - loss: 0.0775 - acc: 0.9719 - val_loss: 0.5925 - val_acc: 0.8189
Epoch 181/200
3914/3914 [==============================] - 0s 26us/step - loss: 0.0773 - acc: 0.9716 - val_loss: 0.6139 - val_acc: 0.8134
Epoch 182/200
3914/3914 [==============================] - 0s 22us/step - loss: 0.0797 - acc: 0.9711 - val_loss: 0.6153 - val_acc: 0.8143
Epoch 183/200
3914/3914 [==============================] - 0s 26us/step - loss: 0.0782 - acc: 0.9742 - val_loss: 0.5843 - val_acc: 0.8199
Epoch 184/200
3914/3914 [==============================] - 0s 22us/step - loss: 0.0779 - acc: 0.9734 - val_loss: 0.6148 - val_acc: 0.8143
Epoch 185/200
3914/3914 [==============================] - 0s 24us/step - loss: 0.0772 - acc: 0.9727 - val_loss: 0.5848 - val_acc: 0.8199
Epoch 186/200
3914/3914 [=======================

array([0.785])

In [109]:
Acc

array([81.6])

In [111]:
F

[0.715,
 0.79,
 0.738,
 0.74,
 0.755,
 0.741,
 0.793,
 0.753,
 0.752,
 0.762,
 0.753,
 0.745,
 0.742,
 0.758,
 0.754,
 0.752,
 0.755,
 0.753,
 0.742,
 0.759,
 0.744,
 0.762,
 0.753,
 0.767,
 0.753,
 0.758,
 0.762,
 0.755,
 0.764,
 0.768,
 0.763,
 0.766,
 0.762,
 0.764,
 0.779,
 0.771,
 0.786,
 0.752,
 0.763,
 0.769,
 0.767,
 0.758,
 0.776,
 0.751,
 0.776,
 0.791,
 0.776,
 0.769]

In [112]:
accuracy

[77.2,
 81.8,
 78.5,
 78.8,
 79.6,
 78.7,
 82.1,
 79.4,
 79.4,
 80.1,
 79.5,
 79.1,
 78.8,
 79.8,
 79.6,
 79.5,
 79.6,
 79.5,
 78.8,
 80.0,
 78.9,
 80.1,
 79.5,
 80.5,
 79.4,
 79.9,
 80.1,
 79.6,
 80.2,
 80.5,
 80.1,
 80.4,
 80.1,
 80.2,
 81.2,
 80.6,
 81.8,
 79.4,
 80.1,
 80.5,
 80.5,
 79.8,
 81.0,
 79.4,
 81.0,
 81.9,
 80.9,
 80.5]